In [2]:
import json
import os
import requests
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [2]:
TAGS_FILE = "testing/test.json"
INPUT_FILE = "../resources/videos-scrap.json"
OUTPUT_FILE = "sources/videos_with_tags.json"
API_URL = "https://models.github.ai/inference/chat/completions"
API_KEY = os.getenv("VITE_OPENAI_API_KEY")
MODEL = "openai/gpt-4.1-nano"

In [3]:
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
tag_store = {}

In [8]:
with open("sources/today.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [9]:
new_tags = []
for item in data:
    for video in item["videos"]:
        if "tag" in video:
            if video['tag'] not in new_tags:
                new_tags.append(video['tag'])

In [10]:
len(new_tags)

0

In [8]:
import time

MAX_RETRIES = 5
WAIT_SECONDS = 5 

In [58]:
with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
    data_helper = json.load(f)

In [12]:
videos_no_tag = []
for item in data:
    for video in item["videos"]:
        if not video.get("tag"):
            videos_no_tag.append(video)

In [13]:
len(videos_no_tag)

87

In [52]:
tag_store

{'Cap-26: Alclad 2, Pinturas Metálicas Especializadas En Modelismo': 'Tutorial',
 'Unboxing X Speed Build Liger Zero Panzer by. BT Model': 'Unboxing',
 'Girls in love- Andreas Dorau (Subtitulos en español)': 'Pop'}

In [14]:
for video in videos_no_tag:
    print(f"Processing video: {video['titulo']}")  

Processing video: CONVERTIRSE en un MILAGRO | Todopoderoso | Relato y Reflexiones
Processing video: PEPE Y TITO ERAN LA ESENCIA DE AFHS | Sobreanalizando AFHS
Processing video: Where The Wind Blows OST | One Piece Film Red | Subtitulado Al Español.
Processing video: Comedy ED.1 l Spy x Family l Subtitulado Al Español.
Processing video: BAD BUNNY MIX ALBUMS - BAD BUNNY PLAYLIST
Processing video: 🔶️EL INMORTAL QUE QUERÍA MOR1R CONOCIÓ A UNA CHICA QUE DESTRUYE TODO LO QUE TOCA (1-7)-Anime Resumen
Processing video: The 1
Processing video: CHARO Y REYNA SE NECESITABAN | Sobreanalizando AFHS
Processing video: La Gata Bajo La Lluvia
Processing video: Esteman - Hasta Que Tú Me Quieras
Processing video: Jaze - QNEPT (Video Oficial)
Processing video: Jaze - caminar al revés (Video Lyric)
Processing video: Científico explica claramente la REALIDAD DEL FENTANILO, sin mentiras ni exageraciones
Processing video: EoO
Processing video: DINA SE VA DE VIAJE | #BRUTALIDADPOLITICA
Processing video: Sabrin

In [3]:
with open('sources/tags.json', 'r', encoding='utf-8') as f:
    data_2 = json.load(f)

In [55]:
def get_tags_from_response(response):
    lines = response.split("\n")
    tags = {}
    for line in lines:
        if "=># " in line:
            try:
                title, tag = line.strip("- ").split("=># ", 1)
                tags[title.strip()] = tag.strip()
                print(f"Title: {title}, Tag: {tag}")
            except ValueError:
                print(f"❌ Línea con formato inválido: {line}")
    return tags

# Leer el contenido del archivo como string completo
with open('sources/helper.md', 'r', encoding='utf-8') as f:
    contenido = f.read()

# Llamar la función con el contenido leído
tags_obtenidos = get_tags_from_response(contenido)

Title: 📌serigrafia con TINTA PUFF de alto relieve🔥parte 2 #viral #estampados #serigrafiatextil #estampado, Tag: Tutorial
Title: Aplicando Técnica de puff alto relieve con plastisol #puf #plastisol #serigrafia, Tag: Tutorial
Title: Todas las legiones de Marines espaciales explicadas! | Warhammer 40k historia (Parte 1/3), Tag: Historia
Title: Introducción a Warhammer 40.000, Tag: Cultura
Title: Cómo hacer una base de diorama oceánica realista | Construyendo el AFV Club U-boot Tipo VII D, Tag: Hobby
Title: Starboy x I Feel It Coming, Tag: Pop
Title: Sunflower, Post Malone, Swae Lee, (Extended Versión), Tag: Pop
Title: El Cadav3r de la Novia : La Historia en 1 Video, Tag: Cine
Title: EL DIA QUE LA TIERRA EXPLOTO | RESUMEN EN 22 MINUTOS, Tag: CienciaFicción
Title: LAS INJUSTICIAS A UN PROFESOR : PELICULA PERUANA, Tag: Drama
Title: América Noticias: Mujer ebria causó accidente de tránsito y protagonizó escándalo en la vía pública, Tag: Informativo
Title: Mitski - My Love Mine All Mine (Exten

In [15]:
titulos = "\n".join(v["titulo"] for v in videos_no_tag)
util_tags = ",".join(new_tags)   

prompt = f"Asigna una etiqueta (tag) temática de una sola palabra para cada uno de los siguientes títulos de video:\n\n{titulos}"

if util_tags:
    prompt += f"\n\nPuedes utiliza las siguientes etiquetas o agregar otras nuevas: {util_tags}"
prompt += "\n\nFormato de respuesta:\n- [Título]=># [Tag]"

print(f"📝 Generando tags para {len(videos_no_tag)} videos...")    
print(f"Prompt: {prompt}")

📝 Generando tags para 87 videos...
Prompt: Asigna una etiqueta (tag) temática de una sola palabra para cada uno de los siguientes títulos de video:

CONVERTIRSE en un MILAGRO | Todopoderoso | Relato y Reflexiones
PEPE Y TITO ERAN LA ESENCIA DE AFHS | Sobreanalizando AFHS
Where The Wind Blows OST | One Piece Film Red | Subtitulado Al Español.
Comedy ED.1 l Spy x Family l Subtitulado Al Español.
BAD BUNNY MIX ALBUMS - BAD BUNNY PLAYLIST
🔶️EL INMORTAL QUE QUERÍA MOR1R CONOCIÓ A UNA CHICA QUE DESTRUYE TODO LO QUE TOCA (1-7)-Anime Resumen
The 1
CHARO Y REYNA SE NECESITABAN | Sobreanalizando AFHS
La Gata Bajo La Lluvia
Esteman - Hasta Que Tú Me Quieras
Jaze - QNEPT (Video Oficial)
Jaze - caminar al revés (Video Lyric)
Científico explica claramente la REALIDAD DEL FENTANILO, sin mentiras ni exageraciones
EoO
DINA SE VA DE VIAJE | #BRUTALIDADPOLITICA
Sabrina Carpenter - Juno (Official Lyric Video)
¿Por qué no hay motos diésel?
🕵 SPY x FAMILY 스파이패밀리 OP/ED FULL모음 (Mixed Nuts, Comedy, Souvenir, 色

In [8]:
with open('sources/today.json', 'r', encoding='utf-8') as f:
    data_today = json.load(f)

In [11]:
next((item["tag"] for item in data_2 if item["title"] == "Yami Safdie, Lasso - En Otra Vida (Video Oficial)"), None)

'Pop'

In [21]:
for item in data_today:
    for video in item["videos"]:
        try :
            video["tag"] = next((item["tag"] for item in data_2 if item["title"] == video['titulo']), None)
        except KeyError:
            print(f"Error: No se encontró la clave 'tag' en el video: {video['titulo']}")
            tag_prov = input(f"¿Qué tag deseas agregar al video '{video['titulo']}'? ")
            video["tag"] = tag_prov
        print(f"Tag '{video['tag']}' agregado al video: {video['titulo']}")

Tag 'Reflexión' agregado al video: CONVERTIRSE en un MILAGRO | Todopoderoso | Relato y Reflexiones
Tag 'Series' agregado al video: PEPE Y TITO ERAN LA ESENCIA DE AFHS | Sobreanalizando AFHS
Tag 'Anime' agregado al video: Where The Wind Blows OST | One Piece Film Red | Subtitulado Al Español.
Tag 'Anime' agregado al video: Comedy ED.1 l Spy x Family l Subtitulado Al Español.
Tag 'Urbano' agregado al video: BAD BUNNY MIX ALBUMS - BAD BUNNY PLAYLIST
Tag 'Anime' agregado al video: 🔶️EL INMORTAL QUE QUERÍA MOR1R CONOCIÓ A UNA CHICA QUE DESTRUYE TODO LO QUE TOCA (1-7)-Anime Resumen
Tag 'Pop' agregado al video: The 1
Tag 'Series' agregado al video: CHARO Y REYNA SE NECESITABAN | Sobreanalizando AFHS
Tag 'Latin' agregado al video: La Gata Bajo La Lluvia
Tag 'Latin' agregado al video: Esteman - Hasta Que Tú Me Quieras
Tag 'Rap' agregado al video: Jaze - QNEPT (Video Oficial)
Tag 'Rap' agregado al video: Jaze - caminar al revés (Video Lyric)
Tag 'Ciencia' agregado al video: Científico explica cl

In [57]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(data_helper, f, ensure_ascii=False, indent=4)
print(f"✅ Etiquetas agregadas y guardadas en {OUTPUT_FILE}")

✅ Etiquetas agregadas y guardadas en sources/videos_with_tags.json


In [ ]:
with open(TAGS_FILE, "r", encoding="utf-8") as f:


In [12]:
for fecha_obj in data:
    fecha = fecha_obj["fecha"]
    videos = fecha_obj["videos"]

    nuevos_videos = [v for v in videos if "tag" not in v]

    if nuevos_videos:
        titulos = "\n".join(v["titulo"] for v in nuevos_videos)
        util_tags = ",".join(new_tags)   

        prompt = f"Asigna una etiqueta (tag) temática de una sola palabra para cada uno de los siguientes títulos de video:\n\n{titulos}"

        if util_tags:
            prompt += f"\n\nPuedes utiliza las siguientes etiquetas o agregar otras nuevas: {util_tags}"
        prompt += "\n\nFormato de respuesta:\n- [Título]=># [Tag]"

        print(f"📝 Generando tags para {len(nuevos_videos)} videos...")
        
        intentos = 0
        while intentos < MAX_RETRIES:
            try:
                response = requests.post(
                    API_URL,
                    headers={
                        "Authorization": f"Bearer {API_KEY}",
                        "Content-Type": "application/json",
                    },
                    json={
                        "model": MODEL,
                        "temperature": 1,
                        "top_p": 1,
                        "messages": [
                            {
                                "role": "system",
                                "content": "Eres un modelo que asigna etiquetas temáticas a títulos de videos musicales.",
                            },
                            {
                                "role": "user",
                                "content": prompt,
                            },
                        ],
                    },
                )

                response.raise_for_status()  # Levanta excepción si código != 200
                content = response.json()["choices"][0]["message"]["content"]
                for line in content.split("\n"):
                    if "=># " in line:
                        try:
                            titulo, tag = line.strip("-").split("=># ", 1)
                            tag_clean = tag.strip().lower()

                            # Comprobar si no está ya en la lista (convertimos todos los elementos a minúsculas para comparar)
                            if tag_clean not in [t.lower() for t in new_tags]:
                                new_tags.append(tag.strip())
                            
                            tag_store[titulo.strip()] = tag.strip()
                        except ValueError:
                            continue

                # Guardar tags actualizados
                with open(TAGS_FILE, "w", encoding="utf-8") as f:
                    json.dump(tag_store, f, indent=2, ensure_ascii=False)

                

                # Recorrer cada objeto en la lista
                for item in data_helper:
                    for video in item["videos"]:
                        titulo = video.get("titulo")  # O la clave real del título
                        if titulo in tag_store:
                                video["tag"] = tag_store[titulo]
                                
                with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
                    json.dump(data_helper, f, indent=2, ensure_ascii=False)
                
                break  # Éxito, salir del while

            except requests.exceptions.HTTPError as e:
                if response.status_code == 429:
                    intentos += 1
                    print(f"⚠️ Rate limit alcanzado. Esperando {WAIT_SECONDS} segundos antes de reintentar (intento {intentos}/{MAX_RETRIES})...")
                    time.sleep(WAIT_SECONDS)
                else:
                    print(f"❌ Error HTTP: {e}")
                    break  # No intentamos más con este error

            except Exception as e:
                print(f"❌ Error inesperado: {e}")
                break

        else:
            print("❌ Se agotaron los reintentos tras error 429. Saltando este prompt.")
            continue

        # Aquí procesas content normalmente y continúas con tu código

        # (Opcional) espera corta para espaciar llamadas
        time.sleep(1)

📝 Generando tags para 23 videos...
📝 Generando tags para 1 videos...
📝 Generando tags para 1 videos...
📝 Generando tags para 13 videos...
📝 Generando tags para 1 videos...
📝 Generando tags para 1 videos...


In [ ]:
with open(TAGS_FILE, "r", encoding="utf-8") as f:
    tags = json.load(f)

In [30]:
with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
    last_try = json.load(f)

In [32]:
for item in last_try:
    for video in item["videos"]:
        if not video.get("tag"):
            print(f"❌ No se encontró tag para el video: {video['titulo']}")

❌ No se encontró tag para el video: 📌serigrafia con TINTA PUFF de alto relieve🔥parte 2 #viral #estampados #serigrafiatextil #estampado
❌ No se encontró tag para el video: HALO-100 (240*320MM)  FORTEVER brand Halo-100 embroidery machine
❌ No se encontró tag para el video: ☑️Configurá #ESLint con #Airbnb y #Prettier.


In [ ]:
tag_store['The Peak']

In [ ]:
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# Recorrer cada objeto en la lista
for item in data:
    if "videos" in item:
        for video in item["videos"]:
            titulo = video.get("titulo")  # O la clave real del título
            if titulo in tag_store:
                video["tag"] = tag_store[titulo]

In [ ]:
data[0]['videos'][0]

In [ ]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"✅ Proceso completado. Resultado guardado en '{OUTPUT_FILE}'")

## META_TAG

In [ ]:
with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [33]:
new_tags = []
for item in data:
    for video in item["videos"]:
        if "tag" in video:
            if video['tag'] not in new_tags:
                new_tags.append(video['tag'])

In [34]:
len(new_tags)

68

In [35]:
new_tags

['Urbano',
 'Debate',
 'Alternativo',
 'Electrónico',
 'Rock',
 'Electro',
 'Lounge',
 'Anime',
 'Pop',
 'Indie',
 'Alternative',
 'Informativo',
 'Comedia',
 'Reflexión',
 'Tutorial',
 'Relajación',
 'Animación',
 'Satírico',
 'Verano',
 'Ciencia',
 'Tecnología',
 'Drama',
 'Sociedad',
 'Motivación',
 'Social',
 'Educación',
 'K-Pop',
 'Latin',
 'Aventura',
 'CienciaFicción',
 'Música',
 'Historia',
 'Cultura',
 'Parodia',
 'Salud',
 'Religión',
 'Hobby',
 'Entretenimiento',
 'Electró',
 'Videojuegos',
 'Acción',
 'Juegos',
 'Rap',
 'Hip-Hop',
 'Unboxing',
 'Viajes',
 'Cine',
 'Jazz',
 'Clásica',
 'Debates',
 'Humor',
 'Dramas',
 'Consejos',
 'Vida',
 'RegionalMexicano',
 'Motociclismo',
 'Dance',
 'Comics',
 'Terror',
 'Fantasía',
 'J-Pop',
 'Documental',
 'Creatividad',
 'Sociedaddebate',
 'Tradicional',
 'Instrumental',
 'Soul',
 'Infomativo']

## Cambiar a la mejor miniatura


In [13]:
import requests

def get_best_thumbnail(video_id):
    base_url = f"https://i.ytimg.com/vi/{video_id}/"
    quality_order = [
        "maxresdefault.jpg",
        "sddefault.jpg",
        "hqdefault.jpg",
        "mqdefault.jpg",
        "default.jpg"
    ]
    
    for quality in quality_order:
        url = base_url + quality
        response = requests.head(url)
        if response.status_code == 200:
            return url  # Primera válida encontrada
    return None  # Ninguna funcionó

def extract_video_id(video_url):
    # Extrae el ID del video desde la URL de YouTube
    if "v=" in video_url:
        return video_url.split("v=")[1].split("&")[0]
    return None

# Ejemplo de uso con tu objeto
video_obj = {
    "titulo": "El Episodio de Spiderman que Muestra Que No Todo Depende de Ti",
    "thumbnail": "https://i.ytimg.com/vi/rRGo9-q_Uhw/hqdefault.jpg",
    "videoUrl": "https://www.youtube.com/watch?v=rRGo9-q_Uhw&t=310s",
    "canal": "Mau Lucero",
    "duracion": "13:12",
    "tag": "Acción"
}

video_id = extract_video_id(video_obj["videoUrl"])

if video_id:
    best_thumb = get_best_thumbnail(video_id)
    if best_thumb:
        video_obj["thumbnail"] = best_thumb
        print(f"✅ Miniatura mejorada encontrada: {best_thumb}")
    else:
        print("❌ No se pudo encontrar una miniatura válida. Usando la original.")
else:
    print("❌ No se pudo extraer el ID del video.")


✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/rRGo9-q_Uhw/maxresdefault.jpg


In [15]:
bad_miniature = []

In [16]:
for item in data_today:
    for video in item["videos"]:
        video_id = extract_video_id(video["videoUrl"])
        if video_id:
            best_thumb = get_best_thumbnail(video_id)
            if best_thumb:
                video["thumbnail"] = best_thumb
                print(f"✅ Miniatura mejorada encontrada: {best_thumb}")
            else:
                bad_miniature.append(video)    
                print("❌ No se pudo encontrar una miniatura válida. Usando la original.")        
        else:
            bad_miniature.append(video)
            print("❌ No se pudo extraer el ID del video.")
with open("sources/today.json", "w", encoding="utf-8") as f:
    json.dump(data_today, f, ensure_ascii=False, indent=4)

✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/5o15fL3HkfY/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/A1M40mwYVzU/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/QWePhaICHMw/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/QWBqXDUasY8/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/yCMEAAkVFd4/sddefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/vPmRtOv_rIk/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/NefIygtTu-8/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/zpF7HQ3tO3U/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/7d9Sfbt3zW0/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/3Be8ub1Vzgg/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/h8TPz6IspCU/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/Zb

In [22]:
data_today

[{'fecha': 'Ayer',
  'videos': [{'titulo': 'CONVERTIRSE en un MILAGRO | Todopoderoso | Relato y Reflexiones',
    'thumbnail': 'https://i.ytimg.com/vi/5o15fL3HkfY/maxresdefault.jpg',
    'videoUrl': 'https://www.youtube.com/watch?v=5o15fL3HkfY&t=939s',
    'canal': 'Farid Dieck',
    'duracion': '23:11',
    'tag': 'Reflexión'},
   {'titulo': 'PEPE Y TITO ERAN LA ESENCIA DE AFHS | Sobreanalizando AFHS',
    'thumbnail': 'https://i.ytimg.com/vi/A1M40mwYVzU/maxresdefault.jpg',
    'videoUrl': 'https://www.youtube.com/watch?v=A1M40mwYVzU',
    'canal': 'Mate en VHS',
    'duracion': '30:08',
    'tag': 'Series'},
   {'titulo': 'Where The Wind Blows OST | One Piece Film Red | Subtitulado Al Español.',
    'thumbnail': 'https://i.ytimg.com/vi/QWePhaICHMw/maxresdefault.jpg',
    'videoUrl': 'https://www.youtube.com/watch?v=QWePhaICHMw',
    'canal': 'Mochi Mochi',
    'duracion': '4:33',
    'tag': 'Anime'},
   {'titulo': 'Comedy ED.1 l Spy x Family l Subtitulado Al Español.',
    'thumbnail

In [23]:
with open("sources/today.json", "w", encoding="utf-8") as f:
    json.dump(data_today, f, ensure_ascii=False, indent=4)

In [47]:
with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [48]:
count = 0
for item in data:
    for video in item["videos"]:
        if not video.get("tag"):
            count += 1

print(f"❌ No se encontró tag para {count} videos")

❌ No se encontró tag para 40 videos


In [42]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [45]:
for item in bad_miniature:
    print(f"❌ No se encontró miniatura para el video: {video['thumbnail']}")

❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://

In [30]:
with open('sources/metatags.json', 'r', encoding='utf-8') as f:
    metatags = json.load(f)
metatags

{'Música': {'color': '#546A76',
  'emoji': '🎶',
  'tags': ['Urbano',
   'Urbano',
   'Pop',
   'Rock',
   'Electro',
   'Electrónico',
   'Lounge',
   'Indie',
   'Alternative',
   'Alternativo',
   'K-Pop',
   'J-Pop',
   'Latin',
   'Jazz',
   'Clásica',
   'Rap',
   'Hip-Hop',
   'Dance',
   'Soul',
   'Instrumental',
   'RegionalMexicano',
   'Tradicional',
   'Electró',
   'Música']},
 'Educación y Conocimiento': {'color': '#88A0A8',
  'emoji': '📚',
  'tags': ['Tutorial',
   'Educación',
   'Historia',
   'Cultura',
   'Ciencia',
   'Tecnología',
   'Documental',
   'Informativo',
   'Infomativo',
   'Consejos',
   'Creatividad']},
 'Entretenimiento': {'color': '#B4CEB3',
  'emoji': '🎉',
  'tags': ['Comedia',
   'Parodia',
   'Humor',
   'Satírico',
   'Unboxing',
   'Cine',
   'Series',
   'Drama',
   'Dramas',
   'Aventura',
   'Acción',
   'Terror',
   'Fantasía',
   'Anime',
   'Animación',
   'Comics',
   'Videojuegos',
   'Juegos',
   'CienciaFicción',
   'Relajación']},
 'R

In [35]:
# Juntar todos los tags en una sola lista
all_tags = []

for categoria in metatags.values():
    all_tags.extend(categoria.get("tags", []))

# Eliminar duplicados (opcional)
all_tags_unicos = list(set(all_tags))

# Mostrar resultado
print(all_tags_unicos)

['Alternativo', 'Lounge', 'Rap', 'Juegos', 'Debate', 'RegionalMexicano', 'Educación', 'Hip-Hop', 'Modelismo Selecto', 'Infomativo', 'Electró', 'Ciencia', 'Satírico', 'Indie', 'Unboxing', 'Tradicional', 'Consejos', 'Urbano', 'Comics', 'Religión', 'Hobby', 'Dance', 'Tecnología', 'Drama', 'Sociedad', 'Salud', 'Informativo', 'CienciaFicción', 'Instrumental', 'Dramas', 'Viajes', 'Sociedaddebate', 'Latin', 'Electrónico', 'Alternative', 'Debates', 'Verano', 'Fantasía', 'Rock', 'Historia', 'Relajación', 'Creatividad', 'Cine', 'Comedia', 'Reflexión', 'Videojuegos', 'Clásica', 'Aventura', 'Electro', 'Animación', 'Jazz', 'K-Pop', 'Series', 'Documental', 'Vida', 'Reflexiones', 'Social', 'Pop', 'Acción', 'Tutorial', 'Cultura', 'Parodia', 'Terror', 'Humor', 'Motivación', 'Soul', 'Música', 'Entretenimiento', 'Motociclismo', 'J-Pop', 'Anime']


In [37]:
for item in data_today:
    for video in item["videos"]:
        if video['tag'] not in all_tags_unicos:
            print(video['tag'])

Cover
